In [ ]:
# !pip install emoji
!pip install bert-embedding
# !pip install mxnet-cu92
# !pip install --upgrade numpy

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import mxnet as mx
from bert_embedding import BertEmbedding

import emoji
import re


In [ ]:
path_prefix='../input/'
TRAIN_FILE = path_prefix + 'train.txt'
DEV_FILE =  path_prefix + 'dev.txt'
TEST_FILE =  path_prefix + 'test.txt'

TURNS_NAMES = ["turn1", "turn2", "turn3"]
LABEL = ["label"]
TURNS_CONCAT = "turns"
def parse_file(file_path):
    output_dict = dict()
    with open(file_path, newline='\n', encoding='utf8') as csvfile:
        return pd.read_csv(csvfile, sep="\t")

train_data = parse_file(TRAIN_FILE)
dev_data = parse_file(DEV_FILE)
test_data = parse_file(TEST_FILE)

In [ ]:
def concatenate_turns(df, delim="fullstop"):
    turns = [("%s %s %s %s %s" %
                 (row[TURNS_NAMES[0]], delim,
                  row[TURNS_NAMES[1]], delim,
                  row[TURNS_NAMES[2]])).lower()
                 for index, row in df.iterrows()]
    df[CONCATENATED_TURNS] = pd.Series(turns, index=df.index)
    return df
def emoticons_replace(df):
    for index, row in df.iterrows():
        for turn in range(3):
          turns = emoji.demojize(row[TURNS_NAMES[turn]])
          # remove delimiters ":"  (:smiley: -> smiley)
          for emoj in re.findall(":\w*:", turns):
              turns  = turns.replace(emoj, emoj[1:-1])
          df.at[index, TURNS_NAMES[turn]] = turns
    return df

In [ ]:
train = concatenate_turns(emoticons_replace(train_data))
dev = concatenate_turns(emoticons_replace(dev_data))
test = concatenate_turns(emoticons_replace(test_data))

In [ ]:
### angry: [1 0 0 0]
### happy: [0 1 0 0]
### others: [0 0 1 0]
### sad: [0 0 0 1]
Y_train = pd.get_dummies(train[LABEL]).as_matrix()
Y_dev = pd.get_dummies(dev[LABEL]).as_matrix()
Y_test = pd.get_dummies(test[LABEL]).as_matrix()

In [ ]:
# ctx = mx.gpu(0)
# bert = BertEmbedding(ctx=ctx)

In [ ]:
# X_train = [bert(train[TURNS_NAMES[i]]) for i in range(3)]
# X_dev = [bert(dev[TURNS_NAMES[i]]) for i in range(3)]
# X_test = [bert(test[TURNS_NAMES[i]]) for i in range(3)]

In [ ]:
# bert_embedding = BertEmbedding(model='bert_12_768_12')

In [ ]:
import os
os.chdir("/kaggle/working/")
pd.DataFrame(X_train).to_csv('test_file')

In [ ]:
# X_train = bert_embedding(train[TURNS_CONCAT][:2])
# print(X_train[:2])
# pd.DataFrame(X_train).to_csv('X_train_bert_2')

In [ ]:
X_train[0][0][0]

In [ ]:
np.shape(X_train[0][0][1])

In [ ]:
def create_model():
    input_layer = Input(shape=(max_sentence,), dtype='int32')

    embedding_layer = Embedding(vocabulary_size,
                                embed_dim,
                                weights=[embedding_matrix],
                                input_length=max_sentence,
                                trainable=False)(input_layer)
    bi_lstm = Bidirectional(LSTM(lstm_out))(embedding_layer)
    dropout = Dropout(0.4)(bi_lstm)
    dense = Dense(128,activation=activation_fct)(dropout)
    dropout = Dropout(0.2)(dense)
    dense = Dense(4,activation=activation_fct)(dropout)
    adam = optimizers.Adam(lr=0.01)
    rmsprop = optimizers.RMSprop(lr=0.005)#, rho=0.9, epsilon=None, decay=0.0)
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

    model = Model(inputs=[input_layer], outputs=dense)
    model.compile(loss = loss_fct, optimizer=adam, metrics = ['accuracy'])
    return model

In [ ]:
# model = KerasClassifier(build_fn = create_model, verbose = 0)
# param_grid = {'batch_size':[10,30, 64, 128, 256, 512, 1024] , 'epochs':[10, 50, 100]}
# grid_search = GridSearchCV(estimator= model, param_grid = param_grid, cv = 5)
# grid_search.fit(X_train, y_train)
# print(grid_search.best_params_)